# Duckdb

Es un sistema de gestión de bases de datos relacionales de código abierto diseñado para consultas analíticas rápidas y eficientes. A menudo se describe como un "SQLite para análisis de datos", ya que es ligero y fácil de integrar, pero al mismo tiempo es muy potente en el procesamiento de grandes volúmenes de datos.

In [ ]:
import duckdb as db

## Generar conexion

In [ ]:
# Conectar a DuckDB
# conn = db.connect(database=":memory:")  # Base en memoria
conn = db.connect(database="../data/database.db")  # Base en físico

In [ ]:
# Crear schema de base de datos
conn.sql("create schema if not exists DATMLOPS")

## Crear tabla 

In [ ]:
# Crear tabla leyendo ficheros parquet
conn.sql(
    """
    create or replace table DATMLOPS.PRUEBA as
    select *
    from read_parquet("../data/prueba2/*.parquet")
    """
)

In [ ]:
# Crear la tabla REGION_X_CIUDAD
conn.sql("""
    create or replace table DATMLOPS.REGION_X_CIUDAD (
        REGION varchar,
        CIUDAD varchar
    )
""")

# Insertar varios registros en una sola sentencia
conn.sql("""
    INSERT INTO DATMLOPS.REGION_X_CIUDAD VALUES
    ('REGION_0', 'CIUDAD_0'),
    ('REGION_1', 'CIUDAD_1'),
    ('REGION_2', 'CIUDAD_2'),
""")

## Ver metadata y descriptivos

In [ ]:
# Metadata
# "columns", "tables", "views", "schemata", "table_constraints", "key_column_usage", "constraint_column_usage"
conn.sql(
    """
    select *
    FROM information_schema.tables
    """
).show()

In [ ]:
# Tipos de variables
conn.sql(
    """
    describe DATMLOPS.PRUEBA
    """
).df().head()

In [ ]:
# Resumen con analisis descriptivo
conn.sql(
    """
    summarize DATMLOPS.PRUEBA
    """
)

## Slicing

In [ ]:
# Seleccionar primeros 5 registros
dbf = conn.sql(
    """
    select *
    from DATMLOPS.PRUEBA
    limit 5 offset 2
    """
)
dbf

## Join

In [ ]:
# join
dfb_vf = conn.sql(
    """
    select *
    from DATMLOPS.PRUEBA
    left join DATMLOPS.REGION_X_CIUDAD
    on COL_A = REGION
    limit 5 offset 0
    """
)
dfb_vf.df()

## Ordenar

In [ ]:
# Ordendar
dbf = conn.sql(
    """
    select *
    from DATMLOPS.PRUEBA
    order by NRO_ID, DATE_FIELD
    limit 5 offset 2
    """
)
dbf

## Escritura

In [ ]:
# Escribir a parquet
conn.sql(
    """
    COPY (SELECT * FROM DATMLOPS.PRUEBA) TO '../data/prueba3.parquet' (FORMAT parquet)
    """
)

In [ ]:
conn.close()